# Compare the pipeline with and without rigid pre-alignment

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import open3d as o3d
import pandas as pd
%load_ext autoreload
%autoreload 2

from DLC_for_WBFM.utils.feature_detection.utils_features import *
from DLC_for_WBFM.utils.feature_detection.utils_tracklets import *
from DLC_for_WBFM.utils.feature_detection.utils_detection import *
from DLC_for_WBFM.utils.feature_detection.visualization_tracks import *
from DLC_for_WBFM.utils.feature_detection.feature_pipeline import *
from DLC_for_WBFM.utils.feature_detection.utils_affine import *
from DLC_for_WBFM.utils.feature_detection.class_reference_frame import PreprocessingSettings
import pickle

In [2]:
# Get the 3d bigtiff folder
bigtiff_folder = r'D:\More-stabilized-wbfm'

btf_fname_red = r'test2020-10-22_16-15-20_test4-channel-0-pco_camera1\test2020-10-22_16-15-20_test4-channel-0-pco_camera1bigtiff.btf'
btf_fname_red = os.path.join(bigtiff_folder, btf_fname_red)

In [7]:
# Same opt for both
opt = {'start_frame':0,
       'num_frames':500,
       'verbose':0}

# old_matches, old_conf, old_frames = track_neurons_full_video(btf_fname_red, **opt)

preprocessing_settings = PreprocessingSettings()
preprocessing_settings.do_filtering=False
preprocessing_settings.do_rigid_alignment=True
preprocessing_settings.do_mini_max_projection=True
new_matches, new_conf, new_frames = track_neurons_full_video(btf_fname_red, **opt, preprocessing_settings=preprocessing_settings)

  4%|███▏                                                                           | 20/499 [04:34<1:59:01, 14.91s/it]

[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.


 98%|████████████████████████████████████████████████████████████████████████████▍ | 489/499 [1:50:50<02:12, 13.24s/it]

[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.


 98%|████████████████████████████████████████████████████████████████████████████▌ | 490/499 [1:51:03<01:58, 13.19s/it]

[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.


 98%|████████████████████████████████████████████████████████████████████████████▋ | 491/499 [1:51:16<01:45, 13.16s/it]

[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.


100%|██████████████████████████████████████████████████████████████████████████████| 499/499 [1:53:02<00:00, 13.59s/it]


In [17]:
clust_df = build_tracklets_from_classes(new_frames, new_matches, new_conf, verbose=0)

In [ ]:
fname = 'clust_df_dat_prealigned.pickle'
with open(fname, 'wb') as f:
    pickle.dump(clust_df,f)
fname = 'match_dat_prealigned.pickle'
with open(fname, 'wb') as f:
    pickle.dump((new_matches, new_conf), f)

In [26]:
fname = 'frame_dat_prealigned.pickle'
[frame.prep_for_pickle() for frame in new_frames]
with open(fname, 'wb') as f:
    pickle.dump(new_frames, f)

In [2]:
# Stitch 
#... Recover if crashed
fname = 'clust_df_dat_prealigned.pickle'
with open(fname, 'rb') as f:
    clust_df = pickle.load(f)
fname = 'frame_dat_prealigned.pickle'
with open(fname, 'rb') as f:
    new_frames = pickle.load(f)

In [3]:
[f.rebuild_keypoints() for f in new_frames];

In [10]:
df, out = stitch_tracklets(clust_df,
                     new_frames,
                     max_stitch_distance=20,
                     min_starting_tracklet_length=20,
                     minimum_match_confidence=0.4,
                     verbose=1)

  0%|                                                                                           | 0/667 [00:00<?, ?it/s]

Trying to consolidate 5835 tracklets
Note: computational time of this function is front-loaded


100%|█████████████████████████████████████████████████████████████████████████████████| 667/667 [38:48<00:00,  3.49s/it]

Extended and dropped 175/5835 rows
Finished


In [11]:
fname = 'clust_df_dat_prealigned-stitched.pickle'
with open(fname, 'wb') as f:
    pickle.dump(df,f)

## TODO: compare neuron tubes

## Visualize the neurons found

In [5]:
which_frame = 1

f_old = old_frames[which_frame]
n_old, pc_old, _ = build_neuron_tree(f_old.neuron_locs)
pc_old.paint_uniform_color([0.5,0.5,0.5])

f_new = new_frames[which_frame]
n_new, pc_new, _ = build_neuron_tree(f_new.neuron_locs)
pc_new.paint_uniform_color([0,1,0])

PointCloud with 137 points.

In [6]:
o3d.visualization.draw_geometries([pc_old, pc_new])

## Visualize the matches

In [4]:
which_pair = (0,1)

old_lines = plot_matched_point_clouds(old_frames,old_matches,which_pair, [0,0,1], False)

In [5]:
new_lines = plot_matched_point_clouds(new_frames,new_matches,which_pair,[0,1,0], False)

In [6]:
o3d.visualization.draw_geometries(old_lines + new_lines)

# Other: save some prealigned data

In [29]:
p = PreprocessingSettings(do_rigid_alignment=True, final_dtype='uint16', alpha=1.0)
# Number of frames
start_frame = 100
num_frames = 100
range_frame = list(range(start_frame, start_frame+num_frames))
# Preallocate numpy array
sz = get_single_volume(btf_fname_red, 0, 33).shape
all_dat = np.zeros((num_frames,)+sz)

In [30]:
for i, i_global in tqdm(enumerate(range_frame), total=len(range_frame)):
    dat_raw = get_single_volume(btf_fname_red, i_global, 33, dtype='uint16')
    all_dat[i,...] = perform_preprocessing(dat_raw, p)

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [05:34<00:00,  3.35s/it]


In [31]:
intermediate_folder = 'test2020-10-22_16-15-20_test4-channel-0-pco_camera1'
out_fname = f'prealigned-vol101-201.tif'
out_fname = os.path.join(bigtiff_folder, intermediate_folder, out_fname)

import tifffile
tifffile.imwrite(out_fname, all_dat)